In [1]:
import cv2
import glob
import numpy as np
import pickle
import scipy.stats
import skimage.feature
import sklearn.model_selection
import sklearn.preprocessing
import sklearn.svm
import time

# Pipeline functions

In [2]:
def get_features_from_paths(car_paths, noncar_paths):
    """Depends on global function `GET_RAW_FEATURES_FROM_IMAGE`,
    which takes one arg of path string and returns one 1-dimensinoal np array of features."""
    
    car_raw_features, noncar_raw_features = [
        [GET_RAW_FEATURES_FROM_IMAGE(path) for path in paths]
        for paths in [car_paths, noncar_paths]
    ]
    
    X = np.vstack((car_raw_features, noncar_raw_features)).astype(np.float64)
    print('pre-split features shape is', X.shape)

    y = np.hstack((np.ones(len(car_raw_features)), np.zeros(len(noncar_raw_features))))

    return X, y

# def get_features_small():
#     car_paths = glob.glob('working_dir/vehicles_smallset/**/*')
#     noncar_paths = glob.glob('working_dir/non-vehicles_smallset/**/*')
#     return get_features_from_paths(car_paths, noncar_paths)

def get_features_large():
    t0 = time.time()
    
    car_paths = glob.glob('working_dir/vehicles/**/*')
    noncar_paths = glob.glob('working_dir/non-vehicles/**/*')
    features = get_features_from_paths(car_paths, noncar_paths)
    
    t1 = time.time()
    print(round(t1 - t0, 2), 'Seconds to collect features')
    
    return features

In [3]:
# A legacy version of training and validating pipeline.
# This 1) searches for an optimal SVC using the small dataset, and
# 2) uses LinearSVC with default parameters on the large dataset, and
# 3) compares their accuracies.
# In the final pipeline, searching is omitted, because it has consistently been much more
# expensive than and has yielded worse models than the latter strategy.
'''
def train_and_validate___legacy():
    """Before running, set global function `GET_RAW_FEATURES_FROM_IMAGE`."""
    
    def train_and_validate_small():
        (X_train, X_test, y_train, y_test), X_scaler = get_classifier_features_small()

        t0 = time.time()

        grid_search_params = {
            'C': scipy.stats.expon(scale=100),
            'gamma': scipy.stats.expon(scale=.1),
            'kernel': ['linear', 'rbf'],
            'class_weight':['balanced', None]}
        clf = sklearn.model_selection.RandomizedSearchCV(sklearn.svm.SVC(), grid_search_params)
        clf.fit(X_train, y_train)

        t1 = time.time()
        
        print(round(t1 - t0, 2), 'Seconds to optimize SVC')
        print('score of optimized classifier on small dataset', clf.score(X_test, y_test))
        print('optimized classifier params', clf.best_params_)

        return X_scaler, clf
    
    def train_and_validate_large(optimized_clf):
        (X_train, X_test, y_train, y_test), X_scaler = get_classifier_features_large()
    
        # it's prob not ok to use a different X_scaler than the classifier was trained for, like in this line here...
        print('score of optimized classifier on large dataset', optimized_clf.score(X_test, y_test))

        t0 = time.time()

        simple_clf = sklearn.svm.LinearSVC()
        simple_clf.fit(X_train, y_train)

        t1 = time.time()
        
        print(round(t1 - t0, 2), 'Seconds to train default LinearSVC')
        print('score of default LinearSVC on large dataset', simple_clf.score(X_test, y_test))

        return X_scaler, simple_clf
    
    X_scaler_small, optimized_clf = train_and_validate_small()
    X_scaler_large, simple_clf = train_and_validate_large(optimized_clf)
    return X_scaler_small, X_scaler_large, optimized_clf, simple_clf
'''

In [3]:
def train_and_validate():
    """Before running, set global function `GET_RAW_FEATURES_FROM_IMAGE`."""
    
    X, y = get_features_large()
    
    X_scaler = sklearn.preprocessing.StandardScaler().fit(X)
    scaled_X = X_scaler.transform(X)

    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
        scaled_X, y, test_size=0.2)
    
    t0 = time.time()
    
    clf = sklearn.svm.LinearSVC()
    clf.fit(X_train, y_train)
    
    t1 = time.time()
    
    print(round(t1 - t0, 2), 'Seconds to train default LinearSVC')
    print('score of default LinearSVC on large dataset', clf.score(X_test, y_test))
    
    return X_scaler, clf

# Extract hog features only

In [4]:
def hog_helper(img_single_ch):
    pixels_per_cell = 8
    cells_per_block = 2
    orientations = 9
    
    return skimage.feature.hog(
        img_single_ch,
        orientations=orientations,
        pixels_per_cell=(pixels_per_cell, pixels_per_cell),
        cells_per_block=(cells_per_block, cells_per_block),
        transform_sqrt=True,
        visualise=False,
        feature_vector=False,
        block_norm='L2-Hys')

def create_hog_features_getter(img_bgr_to_hog_features):
    def getter(path):
        img_bgr = cv2.imread(path)
        assert img_bgr.shape[:2] == (64, 64)

        hog = img_bgr_to_hog_features(img_bgr)
        assert hog.shape[0] == hog.shape[1]

        return hog.flatten()
    
    return getter

### yuv

In [5]:
def get_hog_features_yuv(img_bgr):
    img_yuv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2YUV)
    img_ych = img_yuv[:,:,0]
    return hog_helper(img_ych)

GET_RAW_FEATURES_FROM_IMAGE = create_hog_features_getter(get_hog_features_yuv)

YUV_RESULTS = train_and_validate()

pre-split features shape is (17760, 1764)
85.7 Seconds to collect features
18.0 Seconds to train default LinearSVC
score of default LinearSVC on large dataset 0.918355855856


### hsv

In [6]:
def get_hog_features_hsv(img_bgr):
    img_hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    img_sch = img_hsv[:,:,1]
    img_vch = img_hsv[:,:,2]
    hog_sch = hog_helper(img_sch)
    hog_vch = hog_helper(img_vch)
    return np.stack((hog_sch, hog_vch), axis=-1)

GET_RAW_FEATURES_FROM_IMAGE = create_hog_features_getter(get_hog_features_hsv)

HSV_RESULTS = train_and_validate()

pre-split features shape is (17760, 3528)
77.66 Seconds to collect features
20.66 Seconds to train default LinearSVC
score of default LinearSVC on large dataset 0.956362612613


### hls

In [7]:
def get_hog_features_hls(img_bgr):
    img_hls = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HLS)
    img_lch = img_hls[:,:,1]
    img_sch = img_hls[:,:,2]
    hog_lch = hog_helper(img_lch)
    hog_sch = hog_helper(img_sch)
    return np.stack((hog_lch, hog_sch), axis=-1)

GET_RAW_FEATURES_FROM_IMAGE = create_hog_features_getter(get_hog_features_hls)

HLS_RESULTS = train_and_validate()

pre-split features shape is (17760, 3528)
76.83 Seconds to collect features
19.76 Seconds to train default LinearSVC
score of default LinearSVC on large dataset 0.95411036036


### luv

In [8]:
def get_hog_features_luv(img_bgr):
    img_luv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LUV)
    img_lch = img_luv[:,:,0]
    return hog_helper(img_lch)

GET_RAW_FEATURES_FROM_IMAGE = create_hog_features_getter(get_hog_features_luv)

LUV_RESULTS = train_and_validate()

pre-split features shape is (17760, 1764)
67.86 Seconds to collect features
18.99 Seconds to train default LinearSVC
score of default LinearSVC on large dataset 0.917511261261


### ycrcb

In [9]:
def get_hog_features_ycrcb(img_bgr):
    img_ycrcb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2YCrCb)
    img_ych = img_ycrcb[:,:,0]
    return hog_helper(img_ych)

GET_RAW_FEATURES_FROM_IMAGE = create_hog_features_getter(get_hog_features_ycrcb)

YCRCB_RESULTS = train_and_validate()

pre-split features shape is (17760, 1764)
40.87 Seconds to collect features
18.35 Seconds to train default LinearSVC
score of default LinearSVC on large dataset 0.918355855856


### Pickle

In [11]:
with open('working_dir/hog_training_result.p', 'wb') as f:
    training_result = {
        'YUV_RESULTS': YUV_RESULTS,
        'HSV_RESULTS': HSV_RESULTS,
        'HLS_RESULTS': HLS_RESULTS,
        'LUV_RESULTS': LUV_RESULTS,
        'YCRCB_RESULTS': YCRCB_RESULTS
    }
    pickle.dump(training_result, f)

# Extract hog and color features

In [25]:
def create_color_features_getter(img_bgr_to_color_features, shrunk_dim=24, img_bgr_to_hog_features=get_hog_features_hsv):
    def getter(path):
        img_bgr = cv2.imread(path)
        
        # for hog, hsv is the winner.
        hog = img_bgr_to_hog_features(img_bgr).flatten()
        assert hog.shape == (7 * 7 * 2 * 2 * 9 * 2,)
        
        img_bgr_shrunk = cv2.resize(img_bgr, (shrunk_dim, shrunk_dim))
        clr, num_chs = img_bgr_to_color_features(img_bgr_shrunk)
        assert clr.shape == ((shrunk_dim * shrunk_dim + 32) * num_chs,)

        return np.concatenate((hog, clr))
    
    return getter

### rgb

In [13]:
def get_color_features_rgb(img_bgr_shrunk):
    rch = img_bgr_shrunk[:,:,2]
    
    spatial = rch.flatten()
    
    hist = np.histogram(rch, bins=32, range=(0, 256))
    
    return np.concatenate((spatial, hist[0])), 1

GET_RAW_FEATURES_FROM_IMAGE = create_color_features_getter(get_color_features_rgb)

CLR_RGB_RESULTS = train_and_validate()

pre-split features shape is (17760, 4136)
80.8 Seconds to collect features
16.28 Seconds to train default LinearSVC
score of default LinearSVC on large dataset 0.970439189189


### hsv

In [14]:
def get_color_features_hsv(img_bgr_shrunk):
    img_hsv = cv2.cvtColor(img_bgr_shrunk, cv2.COLOR_BGR2HSV)
    sch = img_hsv[:,:,1]
    
    spatial = sch.flatten()
    
    hist = np.histogram(sch, bins=32, range=(0, 256))
    
    return np.concatenate((spatial, hist[0])), 1

GET_RAW_FEATURES_FROM_IMAGE = create_color_features_getter(get_color_features_hsv)

CLR_HSV_RESULTS = train_and_validate()

pre-split features shape is (17760, 4136)
124.75 Seconds to collect features
18.26 Seconds to train default LinearSVC
score of default LinearSVC on large dataset 0.965371621622


### yuv

In [15]:
def get_color_features_yuv(img_bgr_shrunk):
    img_yuv = cv2.cvtColor(img_bgr_shrunk, cv2.COLOR_BGR2YUV)
    ych = img_yuv[:,:,0]
    
    spatial = ych.flatten()
    
    hist = np.histogram(ych, bins=32, range=(0, 256))
    
    return np.concatenate((spatial, hist[0])), 1

GET_RAW_FEATURES_FROM_IMAGE = create_color_features_getter(get_color_features_yuv)

CLR_YUV_RESULTS = train_and_validate()

pre-split features shape is (17760, 4136)
127.37 Seconds to collect features
16.97 Seconds to train default LinearSVC
score of default LinearSVC on large dataset 0.963119369369


### r and s

In [16]:
def get_color_features_rs(img_bgr_shrunk):
    rch = img_bgr_shrunk[:,:,2]
    
    img_hsv = cv2.cvtColor(img_bgr_shrunk, cv2.COLOR_BGR2HSV)
    sch = img_hsv[:,:,1]
    
    rch_spatial = rch.flatten()
    sch_spatial = sch.flatten()
    rch_hist = np.histogram(rch, bins=32, range=(0, 256))
    sch_hist = np.histogram(sch, bins=32, range=(0, 256))
    
    return np.concatenate((rch_spatial, sch_spatial, rch_hist[0], sch_hist[0])), 2

GET_RAW_FEATURES_FROM_IMAGE = create_color_features_getter(get_color_features_rs)

CLR_RS_RESULTS = train_and_validate()

pre-split features shape is (17760, 4744)
128.5 Seconds to collect features
17.87 Seconds to train default LinearSVC
score of default LinearSVC on large dataset 0.973817567568


### shrink differently

In [17]:
GET_RAW_FEATURES_FROM_IMAGE = create_color_features_getter(get_color_features_rgb, shrunk_dim=32)

CLR_R_SHRUNK32_RESULTS = train_and_validate()

pre-split features shape is (17760, 4584)
128.23 Seconds to collect features
18.64 Seconds to train default LinearSVC
score of default LinearSVC on large dataset 0.968186936937


In [18]:
GET_RAW_FEATURES_FROM_IMAGE = create_color_features_getter(get_color_features_rgb, shrunk_dim=16)

CLR_R_SHRUNK16_RESULTS = train_and_validate()

pre-split features shape is (17760, 3816)
91.27 Seconds to collect features
14.63 Seconds to train default LinearSVC
score of default LinearSVC on large dataset 0.967905405405


In [19]:
GET_RAW_FEATURES_FROM_IMAGE = create_color_features_getter(get_color_features_rs, shrunk_dim=32)

CLR_RS_SHRUNK32_RESULTS = train_and_validate()

pre-split features shape is (17760, 5640)
82.88 Seconds to collect features
22.87 Seconds to train default LinearSVC
score of default LinearSVC on large dataset 0.974943693694


In [21]:
GET_RAW_FEATURES_FROM_IMAGE = create_color_features_getter(get_color_features_rs, shrunk_dim=16)

CLR_RS_SHRUNK16_RESULTS = train_and_validate()

pre-split features shape is (17760, 4104)
77.17 Seconds to collect features
14.47 Seconds to train default LinearSVC
score of default LinearSVC on large dataset 0.977477477477


### Try hls for hog

In [26]:
GET_RAW_FEATURES_FROM_IMAGE = create_color_features_getter(get_color_features_rs, shrunk_dim=16, img_bgr_to_hog_features=get_hog_features_hls)

HOG_LS_CLR_RS_SHRUNK16_RESULTS = train_and_validate()

pre-split features shape is (17760, 4104)
79.2 Seconds to collect features
14.02 Seconds to train default LinearSVC
score of default LinearSVC on large dataset 0.96509009009


### Try hls for color

In [29]:
def get_color_features_ls(img_bgr_shrunk):
    img_hls = cv2.cvtColor(img_bgr_shrunk, cv2.COLOR_BGR2HLS)
    lch = img_hls[:,:,1]
    sch = img_hls[:,:,2]
    
    lch_spatial = lch.flatten()
    sch_spatial = sch.flatten()
    lch_hist = np.histogram(lch, bins=32, range=(0, 256))
    sch_hist = np.histogram(sch, bins=32, range=(0, 256))
    
    return np.concatenate((lch_spatial, sch_spatial, lch_hist[0], sch_hist[0])), 2

GET_RAW_FEATURES_FROM_IMAGE = create_color_features_getter(get_color_features_ls, shrunk_dim=16, img_bgr_to_hog_features=get_hog_features_hls)

HOG_LS_CLR_LS_RESULTS = train_and_validate()

pre-split features shape is (17760, 4104)
128.09 Seconds to collect features
15.97 Seconds to train default LinearSVC
score of default LinearSVC on large dataset 0.971283783784


In [30]:
def get_color_features_rls(img_bgr_shrunk):
    rch = img_bgr_shrunk[:,:,2]
    
    img_hls = cv2.cvtColor(img_bgr_shrunk, cv2.COLOR_BGR2HLS)
    lch = img_hls[:,:,1]
    sch = img_hls[:,:,2]
    
    rch_spatial = rch.flatten()
    lch_spatial = lch.flatten()
    sch_spatial = sch.flatten()
    rch_hist = np.histogram(rch, bins=32, range=(0, 256))
    lch_hist = np.histogram(lch, bins=32, range=(0, 256))
    sch_hist = np.histogram(sch, bins=32, range=(0, 256))
    
    return np.concatenate((rch_spatial, lch_spatial, sch_spatial, rch_hist[0], lch_hist[0], sch_hist[0])), 3

GET_RAW_FEATURES_FROM_IMAGE = create_color_features_getter(get_color_features_rls, shrunk_dim=16, img_bgr_to_hog_features=get_hog_features_hls)

HOG_LS_CLR_RLS_RESULTS = train_and_validate()

pre-split features shape is (17760, 4392)
130.0 Seconds to collect features
14.1 Seconds to train default LinearSVC
score of default LinearSVC on large dataset 0.97240990991


### Pickle

In [31]:
with open('working_dir/overall_training_result.p', 'wb') as f:
    overall_training_result = {
        'CLR_RGB_RESULTS': CLR_RGB_RESULTS,
        'CLR_HSV_RESULTS': CLR_HSV_RESULTS,
        'CLR_YUV_RESULTS': CLR_YUV_RESULTS,
        'CLR_RS_RESULTS': CLR_RS_RESULTS,
        'CLR_R_SHRUNK32_RESULTS': CLR_R_SHRUNK32_RESULTS,
        'CLR_R_SHRUNK16_RESULTS': CLR_R_SHRUNK16_RESULTS,
        'CLR_RS_SHRUNK32_RESULTS': CLR_RS_SHRUNK32_RESULTS,
        'CLR_RS_SHRUNK16_RESULTS': CLR_RS_SHRUNK16_RESULTS,
        'HOG_LS_CLR_RS_SHRUNK16_RESULTS': HOG_LS_CLR_RS_SHRUNK16_RESULTS,
        'HOG_LS_CLR_LS_RESULTS': HOG_LS_CLR_LS_RESULTS,
        'HOG_LS_CLR_RLS_RESULTS': HOG_LS_CLR_RLS_RESULTS
    }
    pickle.dump(overall_training_result, f)